<a href="https://colab.research.google.com/github/MazinMukhtar/Efficient-Attention-Computation/blob/main/development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Efficient Computation of Attention Using Tesla GPUs

Researching optimization techniques for attention computation in a production ML pipeline using JAX and xFormers for deployment on Tesla GPUs. This notebook will be used to test techniques using the SimCLR model architecture on the ImageNet dataset.